In [1]:
%pip install beautifulsoup4 requests 


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\ShahinGanbar\QSS\Week8_Day1_CS\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
from bs4 import BeautifulSoup
import requests


url = "https://bina.az/"


headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/117.0"
}

response = requests.get(url)

In [ ]:
import requests
import pandas as pd

url = "https://bina.az/graphql"

payload = {
    "operationName": "InitialItems",
    "variables": {},  
    "extensions": {
        "persistedQuery": {
            "version": 1,
            "sha256Hash": "0398d102f7eacb376914f9f039bc8139726d38d913fd1bab5b6ffe0ca56a789c"
        }
    }
}

headers = {
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://bina.az/",
    "Origin": "https://bina.az"
}

response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    data = response.json()
    residences = data.get("data", {}).get("residences", [])

    
    locations = []
    links = []
    published_dates = []
    prices = []

    for res in residences:
        
        location = f"{res.get('address', '')} ({res.get('shortAddress', '')})"
        locations.append(location)

        
        
        link_path = res.get("url")
        if link_path:
            link = "https://bina.az" + link_path
        else:
            
            link = f"https://bina.az/residence/{res.get('id', '')}"
        links.append(link)

        
        published_date = res.get("publishedAt") or res.get("deadline") or "N/A"
        published_dates.append(published_date)

        
        price_info = res.get("price", {})
        currency = price_info.get("currency", "AZN")
        per_unit = price_info.get("perUnit", None)
        total_price = price_info.get("total", {})
        price_str = ""

        if per_unit:
            price_str = f"{per_unit} {currency} per unit"
        elif total_price:
            min_price = total_price.get("minimum")
            max_price = total_price.get("maximum")
            if min_price and max_price:
                price_str = f"{min_price} - {max_price} {currency}"
            elif min_price:
                price_str = f"{min_price} {currency}"
            else:
                price_str = "N/A"
        else:
            price_str = "N/A"

        prices.append(price_str)

    
    df = pd.DataFrame({
        "Location": locations,
        "Link": links,
        "Published Date": published_dates,
        "Price": prices
    })

    print(df)

else:
    print(f"Failed to fetch data. Status code: {response.status_code}")


                                            Location  \
0  Tbilisi prospekti ilə Azər Əliyev küçəsinin kə...   
1  S. Əsgərova, R. Rza, Ə. Topçubaşov və Ş. Əzizb...   
2  Yasamal r., B.Məcidov, A.Şaiq və F.İbrahimov k...   
3            Nəsimi r., Neftçilər pr. 64 (Nəsimi r.)   

                            Link  Published Date              Price  
0  https://bina.az/residence/331      30.09.2027  4800 AZN per unit  
1  https://bina.az/residence/310  Təhvil verilib  3000 AZN per unit  
2  https://bina.az/residence/307            2025  2730 AZN per unit  
3  https://bina.az/residence/322  Təhvil verilib  9642 AZN per unit  
